In [1]:
import altair as alt
import pandas as pd

from fairness_utils import *
from latextable import draw_latex
from texttable import Texttable
from scorer import *

FILENAME = 'data/compas_predictions.csv'
PREDICTOR_TYPES = ['unconstrained','calibrated','thresholdless_EO']
RACES = ['all','African-American','Caucasian']

In [2]:
df = pd.read_csv(FILENAME)

In [3]:
# Fairness and accuracy criteria
table = Texttable()
table.set_deco(Texttable.HEADER)
table.set_cols_dtype([
    't',
    't',
    'f',
    'f',
    'f',
    'f',
    'f',
])
table.add_row(['Predictor','Race','Brier Score','Log Loss','Calibration Gap','Separation Gap (Negative Class)','Separation Gap (Positive Class)'])
brier_accuracy_scorer = WeightedBrierScorer()
log_loss_scorer = WeightedLogScorer()
for predictor in PREDICTOR_TYPES:
    brier_scores = brier_accuracy_scorer.score_many(df['two_year_recid'],df[predictor],df['race'])
    log_losses = log_loss_scorer.score_many(df['two_year_recid'],df[predictor],df['race'])
    calibration_gaps = get_sufficiency_gap(df['two_year_recid'],df[predictor],df['race'])
    sep_neg = get_separation_gap(df['two_year_recid'],df[predictor],df['race'],0)
    sep_pos = get_separation_gap(df['two_year_recid'],df[predictor],df['race'],1)
    for race in RACES:
        if race != 'all':
            table.add_row([predictor,race,brier_scores[race],log_losses[race],calibration_gaps[race],
                       sep_neg[race],sep_pos[race]])
        else:
            table.add_row([predictor,race,brier_scores[race],log_losses[race],float('nan'),float('nan'),float('nan')])
print(table.draw())

Predictor   Race        Brier       Log Loss   Calibrati   Separatio   Separatio
                        Score                  on Gap      n Gap       n Gap    
                                                           (Negative   (Positive
                                                           Class)      Class)   
unconstra   all         0.210       0.986      nan         nan         nan      
ined                                                                            
unconstra   African-    0.211       0.985      nan         0.040       0.034    
ined        American                                                            
unconstra   Caucasian   0.207       0.989      nan         -0.042      -0.071   
ined                                                                            
calibrate   all         0.210       0.974      nan         nan         nan      
d                                                                               
calibrate   African-    0.21

In [4]:
# for latex :~) 
draw_latex(table)

'\\begin{table}\n\t\\begin{center}\n\t\t\\begin{tabular}{lllllll}\n\t\t\t \\\\\n\t\t\t\\hline\n\t\t\tPredictor & Race & Brier Score & Log Loss & Calibration Gap & Separation Gap (Negative Class) & Separation Gap (Positive Class) \\\\\n\t\t\tunconstrained & all & 0.210 & 0.986 & nan & nan & nan \\\\\n\t\t\tunconstrained & African-American & 0.211 & 0.985 & nan & 0.040 & 0.034 \\\\\n\t\t\tunconstrained & Caucasian & 0.207 & 0.989 & nan & -0.042 & -0.071 \\\\\n\t\t\tcalibrated & all & 0.210 & 0.974 & nan & nan & nan \\\\\n\t\t\tcalibrated & African-American & 0.212 & 0.972 & nan & 0.039 & 0.034 \\\\\n\t\t\tcalibrated & Caucasian & 0.207 & 0.978 & nan & -0.042 & -0.069 \\\\\n\t\t\tthresholdless_EO & all & 0.215 & 0.949 & nan & nan & nan \\\\\n\t\t\tthresholdless_EO & African-American & 0.220 & 0.940 & nan & 0.000 & -0.000 \\\\\n\t\t\tthresholdless_EO & Caucasian & 0.208 & 0.961 & nan & -0.000 & 0.000 \\\\\n\t\t\\end{tabular}\n\t\\end{center}\n\\end{table}'

In [5]:
# Heatmap of scores for different Brier, log loss, and Beta weights
# https://altair-viz.github.io/gallery/simple_heatmap.html
